# Graph Loader for Neo4j

Authors: Ganesh Nagarajan and Siddarth Jayshankar

This script reads the file subsetted from the hive and loads the Ne04j graph database. Every Paper is considered as a node and the relationship CITED_BY is created between these two nodes.

### Columns Metadata

In [1]:
columns=['cited_id1','citing_f_name','citing_key_name','citing_auth_name','citing_id','citing_norm_title','cited_f_name','cited_key_name','cited_auth_name','cited_id','cited_norm_title']

## Imports

In [2]:
from neo4jrestclient.client import GraphDatabase
import json

Function Check node checks if there is already a node in the graph database with the given query id and returns true or false. Create Node creates the new node in the graph database

In [21]:
def checknode(connection,node):
    query='match(n {id:"'+node+'"}) return(n)'
    result = connection.query(q=query)
    if len(result)>0:
        return True
    else:
        return False

def createNode(connection,params):
    query='create(n {'
    for (k,v) in params.items():
        query=query+k+':"'+v+'",'
    query=query[:-1]+'})'
    result = connection.query(q=query)

In [4]:
graphdata=[]

Process the graph file list of lists

In [5]:
graph=open('graph.csv','r')
for line in graph.readlines():
    graphdata.append(dict(zip(columns,line.strip().split(","))))

Define Connection Parameters

In [6]:
gdb=GraphDatabase("http://localhost:7474/db/data/",username="neo4j",password="password")

For every relation in the csv, check if both the citing paper and the cited paper are present. If both are present, just create the relationship edge. Else create the missing nodes and create the relationship.

In [25]:
for i in range(0,len(graphdata)-1):
    data=graphdata[i]
    checkfrom=checknode(gdb,data['citing_id'])
    checkto=checknode(gdb,data['cited_id'])
    if not checkfrom:
        write={'id':data['citing_id'],'title':data['citing_norm_title'],'author':data['citing_auth_name'],
               'field':data['citing_f_name'],'keyword':data['citing_key_name']}
        createNode(gdb,write)
    if not checkto:
        write={'id':data['cited_id'],'title':data['cited_norm_title'],'author':data['cited_auth_name'],
               'field':data['cited_f_name'],'keyword':data['cited_key_name']}
        createNode(gdb,write)
    query='MATCH (a),(b) WHERE a.id = "'+data['citing_id']+'" AND b.id = "'+data['cited_id']+'" CREATE (a)-[r:CITEDBY]->(b) RETURN r'
    result = gdb.query(q=query)
         